# Reshaping and Merging DataFrames

## In this section, we'll cover

* Learn about data reshaping
* What wide and long data formats are, and how to convert between them with `pivot` and `melt`
* How to merge data tables together with `concat` and `merge`

## What is Data Reshaping?

Real world data comes in all shapes and sizes, and it's not uncommon to encounter data that's poorly organized in ways that makes it difficult to work with.  Therefore, learning how to reorganize data is a critical skill for anyone working with data.  This process is generally referred to as data reshaping.

While there are all kinds of strange and interesting ways in which data sets can formatted, there are some common organization patterns you'll often encounter.  Two that we'll discuss here are *wide* and *long* forms of tabular data.

Let's first illustrate this with an example of a data table in long format -- the Gapminder dataset (from https://www.gapminder.org, tidied by Jennifer Bryan https://github.com/jennybc).  This is a very interesting data set that looks at life expectancy and per capita GDP for countries across time.

In [1]:
import pandas as pd

# This is actually a tab separated file (not comma); still use read_csv, but specify the tab separator
gapminder = pd.read_csv("https://raw.githubusercontent.com/jennybc/gapminder/master/inst/extdata/gapminder.tsv", sep="\t")
gapminder.head(10)

,country,continent,year,lifeExp,pop,gdpPercap
0,Afghanistan,Asia,1952,28.801,8425333,779.445314
1,Afghanistan,Asia,1957,30.332,9240934,820.853030
2,Afghanistan,Asia,1962,31.997,10267083,853.100710
3,Afghanistan,Asia,1967,34.020,11537966,836.197138
4,Afghanistan,Asia,1972,36.088,13079460,739.981106
5,Afghanistan,Asia,1977,38.438,14880372,786.113360
6,Afghanistan,Asia,1982,39.854,12881816,978.011439
7,Afghanistan,Asia,1987,40.822,13867957,852.395945
8,Afghanistan,Asia,1992,41.674,16317921,649.341395
9,Afghanistan,Asia,1997,41.763,22227415,635.341351


The term *long* refers to the notion that this data long (or tall) in terms of the number of rows vs. the number of columns (thought this does not need to be strictly true). If we were to add additional measurements to this table (e.g. data for a new country or a new year), it would get even longer because we'd need to add additional rows.

## Reshaping Long Data to Wide Data

In the table above, measurements for each country have been made across time.  In this case, there is one row for each country and year combination such that each country is repeated across multiple rows (one for each year), and each year is also repeated across multiple rows (one for each country).

Imagine you were interested in how the life expectancy for each country changes across time.  This data could be reshaped to present this data in a wide form:

In [2]:
gm_life_exp_wide = gapminder.pivot(index = "country", columns="year", values="lifeExp").reset_index()
gm_life_exp_wide.head(10)

year,country,1952,1957,1962,1967,1972,1977,1982,1987,1992,1997,2002,2007
0,Afghanistan,28.801,30.332,31.997,34.020,36.088,38.438,39.854,40.822,41.674,41.763,42.129,43.828
1,Albania,55.230,59.280,64.820,66.220,67.690,68.930,70.420,72.000,71.581,72.950,75.651,76.423
2,Algeria,43.077,45.685,48.303,51.407,54.518,58.014,61.368,65.799,67.744,69.152,70.994,72.301
3,Angola,30.015,31.999,34.000,35.985,37.928,39.483,39.942,39.906,40.647,40.963,41.003,42.731
4,Argentina,62.485,64.399,65.142,65.634,67.065,68.481,69.942,70.774,71.868,73.275,74.340,75.320
5,Australia,69.120,70.330,70.930,71.100,71.930,73.490,74.740,76.320,77.560,78.830,80.370,81.235
6,Austria,66.800,67.480,69.540,70.140,70.630,72.170,73.180,74.940,76.040,77.510,78.980,79.829
7,Bahrain,50.939,53.832,56.923,59.923,63.300,65.593,69.052,70.750,72.601,73.925,74.795,75.635
8,Bangladesh,37.484,39.348,41.216,43.453,45.252,46.923,50.009,52.819,56.018,59.412,62.013,64.062
9,Belgium,68.000,69.240,70.250,70.940,71.440,72.800,73.930,75.350,76.460,77.530,78.320,79.441


In the above table, the years have now been *pivotted* into individual columns and each cell represents a life expectancy for a given country.  The data table has now become wider (horizontally) than it was previously, where the number of columns is defined by the number of unique values that the year column had in the original form of the data.  If we were to add additional year measurements to this table, it would get wider by adding more year columns.

Here we used the `pandas` `pivot` function to accomplish the long to wide reformatting.  This function takes three main arguments:
* `index`: the column that identifies what each row will be identified by, in this case the country -- each row gives measuremtns for a specific country
* `columns`: the column that identifies the values that will *pivot* into the new column names; this will typically be categorical type column
* `values`: the column that identifies which data values will go into the reshaped table

Also, note the use of the `reset_index` function above.  This function takes the row index produced by the `pivot` function (`country` in this case) and makes it a new column as we can see in the output.  We'll need this column in the example below.

Getting these function arguments right can be tricky in the beginning, so don't get too frustrated.  Expertise will come with practice, and pivotting is such a common task that you'll find plenty of opportunities to pratice.

## Reshaping Wide Data to Long Data

Reformatting data from wide to long format can be accomplished using the `melt` function.  Think of the wide data table *melting* down into a longer table:

In [3]:
gm_life_exp_long = gm_life_exp_wide.melt(id_vars="country")
gm_life_exp_long.head(10)

,country,year,value
0,Afghanistan,1952,28.801
1,Albania,1952,55.230
2,Algeria,1952,43.077
3,Angola,1952,30.015
4,Argentina,1952,62.485
5,Australia,1952,69.120
6,Austria,1952,66.800
7,Bahrain,1952,50.939
8,Bangladesh,1952,37.484
9,Belgium,1952,68.000


This data table is now longer (has more rows), and looks like a subset of the original data table that only has the life expectancy data.

The `melt` function takes all the columns *not specified* in the id_vars argument, and collapses them down to two new columns, one representing the new variable names, and the other representing their corresponding values.  In this particular case, we used the `country` column as the id variable, so the `melt` function collected all the remaining column names, the years, and collapsed them down in to a single column called `year` (this name carried over from the `pivot` step).  All the values in the table, the life expectancies, then got gathered up into a new single column with a default name of `value`.  These new column names can also be defined with additional function arguments:

In [4]:
gm_life_exp_wide.melt(id_vars="country", var_name = "my_year", value_name = "lifeExp").head(10)

,country,my_year,lifeExp
0,Afghanistan,1952,28.801
1,Albania,1952,55.230
2,Algeria,1952,43.077
3,Angola,1952,30.015
4,Argentina,1952,62.485
5,Australia,1952,69.120
6,Austria,1952,66.800
7,Bahrain,1952,50.939
8,Bangladesh,1952,37.484
9,Belgium,1952,68.000


## Combining Separate Tables into One Using `concat`

It's often useful to combine multiple data tables into a single, larger table, for example when you want to merge tables with similar structure that exist across multipe files, or merge together data from two different sources.  Generally speaking, combining tables occurs in two ways: stacking two tables "on top" of each other (combining table rows), or smashing two table together (combining table columns).  

Let's first look at the first case: stacking two tables on top of each other.  Note that in general, the two table should have the exact same column names.  For this example, we'll make two separate tables from our source table:

In [5]:
us_data = gapminder[gapminder["country"] == "United States"].reset_index(drop = True)
us_data

,country,continent,year,lifeExp,pop,gdpPercap
0,United States,Americas,1952,68.440,157553000,13990.48208
1,United States,Americas,1957,69.490,171984000,14847.12712
2,United States,Americas,1962,70.210,186538000,16173.14586
3,United States,Americas,1967,70.760,198712000,19530.36557
4,United States,Americas,1972,71.340,209896000,21806.03594
5,United States,Americas,1977,73.380,220239000,24072.63213
6,United States,Americas,1982,74.650,232187835,25009.55914
7,United States,Americas,1987,75.020,242803533,29884.35041
8,United States,Americas,1992,76.090,256894189,32003.93224
9,United States,Americas,1997,76.810,272911760,35767.43303


In [6]:
japan_data = gapminder[gapminder["country"] == "Japan"].reset_index(drop = True)
japan_data

,country,continent,year,lifeExp,pop,gdpPercap
0,Japan,Asia,1952,63.030,86459025,3216.956347
1,Japan,Asia,1957,65.500,91563009,4317.694365
2,Japan,Asia,1962,68.730,95831757,6576.649461
3,Japan,Asia,1967,71.430,100825279,9847.788607
4,Japan,Asia,1972,73.420,107188273,14778.786360
5,Japan,Asia,1977,75.380,113872473,16610.377010
6,Japan,Asia,1982,77.110,118454974,19384.105710
7,Japan,Asia,1987,78.670,122091325,22375.941890
8,Japan,Asia,1992,79.360,124329269,26824.895110
9,Japan,Asia,1997,80.690,125956499,28816.584990


Next, we can use the `concat` function to *concatenate* these two individual tables together:

In [10]:
# Combine the USA and Japan data tables by row
# NOTE: the two tables need to be put inside a Python list, []
us_japan_data = pd.concat([us_data, japan_data]).reset_index(drop = True)
us_japan_data

,country,continent,year,lifeExp,pop,gdpPercap
0,United States,Americas,1952,68.440,157553000,13990.482080
1,United States,Americas,1957,69.490,171984000,14847.127120
2,United States,Americas,1962,70.210,186538000,16173.145860
3,United States,Americas,1967,70.760,198712000,19530.365570
4,United States,Americas,1972,71.340,209896000,21806.035940
5,United States,Americas,1977,73.380,220239000,24072.632130
6,United States,Americas,1982,74.650,232187835,25009.559140
7,United States,Americas,1987,75.020,242803533,29884.350410
8,United States,Americas,1992,76.090,256894189,32003.932240
9,United States,Americas,1997,76.810,272911760,35767.433030


By default, the concat function combines the tables by row, but it's possible to also combine tables by column using the function argument `axis = 1`, (where `1` = "by column", and `0` = "by row").  Concatenating two tables by column "smashes" them together, putting the columns together side by side.  In this case, the column names in the two tables should be different (duplicating column names is a generally bad idea), while the number of rows should be the same.

For this example, we'll create a new table with mock data to add on to the United States data table:

In [8]:
new_data_dict = {"new_col_1": list(range(0, us_data.shape[0])),
                 "new_col_2": list(range(0, us_data.shape[0] * 2, 2))}
new_data = pd.DataFrame(new_data_dict)
new_data

,new_col_1,new_col_2
0,0,0
1,1,2
2,2,4
3,3,6
4,4,8
5,5,10
6,6,12
7,7,14
8,8,16
9,9,18


In [9]:
pd.concat([us_data, new_data], axis = 1)

,country,continent,year,lifeExp,pop,gdpPercap,new_col_1,new_col_2
0,United States,Americas,1952,68.440,157553000,13990.48208,0,0
1,United States,Americas,1957,69.490,171984000,14847.12712,1,2
2,United States,Americas,1962,70.210,186538000,16173.14586,2,4
3,United States,Americas,1967,70.760,198712000,19530.36557,3,6
4,United States,Americas,1972,71.340,209896000,21806.03594,4,8
5,United States,Americas,1977,73.380,220239000,24072.63213,5,10
6,United States,Americas,1982,74.650,232187835,25009.55914,6,12
7,United States,Americas,1987,75.020,242803533,29884.35041,7,14
8,United States,Americas,1992,76.090,256894189,32003.93224,8,16
9,United States,Americas,1997,76.810,272911760,35767.43303,9,18


## Merging Tables Together Using `merge`

Many types of data problems require combining data from multiple sources.  `concat` can be used to perform simple merging (by rows or columns) as we saw above, but more complex merging can be performed using the `merge` function.  `merge` allows for database-like merging of tables, allowing you to specify exactly how they'd like the two tables to be combined.

As an example, consider our GapMinder data set and that we have data from another source for each country that we want to add to our table.  For simplicity, we'll just work with the United States and Japan table we made above, and work with a data table that has the average temperatures of the two countries for a few years.

In [18]:
temperature_dict = {"country": ["United States", "United States", "United States", "Japan", "Japan", "Japan"],
                    "year": [1997, 2002, 2007, 1997, 2002, 2007],
                    "avg_temperature_f": [53, 56, 56, 59, 58, 58]}
temperature_df = pd.DataFrame(temperature_dict)
temperature_df

,country,year,avg_temperature_f
0,United States,1997,53
1,United States,2002,56
2,United States,2007,56
3,Japan,1997,59
4,Japan,2002,58
5,Japan,2007,58


When merging data, it's important to consider which columns correspond to each other between the two tables you want to merge.  In this case, we can see that country and year are common between our tables, and importantly for the merge we are going to do, the values also correspond (i.e. the countries are United States and Japan, and the years are 1997, 2002, and 2007).

In [23]:
merged_df = pd.merge(us_japan_data, temperature_df, how="left", on=["country", "year"])

,country,continent,year,lifeExp,pop,gdpPercap,avg_temperature_f
0,United States,Americas,1952,68.440,157553000,13990.482080,NaN
1,United States,Americas,1957,69.490,171984000,14847.127120,NaN
2,United States,Americas,1962,70.210,186538000,16173.145860,NaN
3,United States,Americas,1967,70.760,198712000,19530.365570,NaN
4,United States,Americas,1972,71.340,209896000,21806.035940,NaN
5,United States,Americas,1977,73.380,220239000,24072.632130,NaN
6,United States,Americas,1982,74.650,232187835,25009.559140,NaN
7,United States,Americas,1987,75.020,242803533,29884.350410,NaN
8,United States,Americas,1992,76.090,256894189,32003.932240,NaN
9,United States,Americas,1997,76.810,272911760,35767.433030,53.0


There are several things to note about the above `merge`:

- We didn't wrap the two tables to merge into a Python list (like we did for `concat`)
- The `how` argument specifies how we want to merge, in this case, using the left table (i.e. the first one specified in the arguments) as the basis for the merge; all rows from the left table will appear in the output of the merge
- The `on` argument gives the names of the columns we want to use for determining how to do the merge;  in this case we want to merge using the values from the `country` and `year` columns
- Since we joined using the left table (`us_japan_data`), all of the rows in that table appear in the output
- Since the other table only included year values for 1997, 2002, and 2007, we only have temperature values for those years in the output; the rest are missing values (`NaN`)

For a different kind of merge, let's see what happens if we merge using the *right* table as the basis for the merge:

In [25]:
merged_df_2 = pd.merge(us_japan_data, temperature_df, how="right", on=["country", "year"])
merged_df_2

,country,continent,year,lifeExp,pop,gdpPercap,avg_temperature_f
0,United States,Americas,1997,76.810,272911760,35767.43303,53
1,United States,Americas,2002,77.310,287675526,39097.09955,56
2,United States,Americas,2007,78.242,301139947,42951.65309,56
3,Japan,Asia,1997,80.690,125956499,28816.58499,59
4,Japan,Asia,2002,82.000,127065841,28604.59190,58
5,Japan,Asia,2007,82.603,127467972,31656.06806,58


Now, we only have 3 rows for each country in the output.  Since we used the right table (`temperature_df`) as the basis for the merge, only values that are present in that table remain after the merge.  These row correspond to the three years (1997, 2002, and 2007) that we had data for in the `temperature_df` table.

Merging data is a deep (and complex) topic that goes well beyond the simple examples above, but is also a powerful tool and worth learning about in more detail.

## Main Points

* Reshaping data is the process of reorganizing how data are arranged, and is often needed to make data analysis tasks easier to perform
* Long format data has data structured in a vertical-centric form (i.e. along rows)
* Wide format data has data structured in a horizontal-centric form (i.e. along columns)
* You can convert between long and wide forms using the `pivot` and `melt` functions
* Merging individual tables together can be accomplished with the `concat` and `merge` functions.  Simple concatenation of row or columns is achieved with `concat`.  Use the `merge` function for more complex joining.